In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from sklearn.model_selection import train_test_split
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/xalan-2.4.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,xalan,2.4.0,org.apache.xalan.xsltc.runtime.output.SAXXMLOu...,16,3,0,6,52,112,1,...,1.000,0,0.740000,0.344444,2,8,30.625000,1,0.8125,0
1,xalan,2.4.0,org.apache.xpath.operations.Operation,12,2,14,23,23,0,17,...,1.000,2,0.731707,0.218750,1,7,11.250000,5,1.5833,0
2,xalan,2.4.0,org.apache.xalan.xsltc.runtime.BasisLibrary,53,1,0,34,150,1362,22,...,0.375,0,0.000000,0.125000,0,0,37.962264,49,4.3962,1
3,xalan,2.4.0,org.apache.xpath.objects.XString,49,3,2,43,100,1176,34,...,0.000,1,0.606838,0.122222,3,21,14.142857,16,1.8571,0
4,xalan,2.4.0,org.apache.xalan.xsltc.StripFilter,1,1,0,5,1,0,5,...,0.000,0,0.000000,1.000000,0,0,0.000000,1,1.0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,xalan,2.4.0,org.apache.xpath.patterns.NodeTestFilter,1,1,0,1,1,0,0,...,0.000,0,0.000000,1.000000,0,0,0.000000,1,1.0000,0
719,xalan,2.4.0,org.apache.xalan.xsltc.compiler.LangCall,4,4,0,19,17,0,0,...,1.000,2,0.969697,0.375000,3,8,19.500000,2,1.0000,0
720,xalan,2.4.0,org.apache.xalan.xsltc.dom.FilterIterator,9,2,0,5,24,0,1,...,1.000,2,0.555556,0.288889,1,5,10.444444,3,1.2222,0
721,xalan,2.4.0,org.apache.xalan.xsltc.compiler.RelativePathPa...,1,5,3,7,2,0,6,...,0.000,0,1.000000,1.000000,0,0,3.000000,0,0.0000,0


In [4]:
train, test = train_test_split(ant_master, test_size=0.2)

In [5]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [6]:
x_train

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
245,7,2,0,11,24,5,1,11,6,0.583333,131,1.0,2,0.900000,0.238095,1,5,17.428571,6,1.8571
344,4,3,0,13,14,0,0,13,4,0.333333,44,1.0,0,0.959459,0.400000,2,7,9.750000,1,0.7500
473,2,2,0,7,9,1,0,7,2,2.000000,39,0.0,0,1.000000,0.750000,0,0,18.500000,0,0.0000
282,5,3,0,8,12,8,1,7,5,0.500000,59,1.0,0,0.904762,0.400000,1,6,10.600000,2,1.0000
51,2,5,0,7,8,1,0,7,2,2.000000,34,0.0,0,0.982456,0.750000,1,6,16.000000,1,0.5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,33,1,0,3,73,0,1,3,33,0.750000,469,0.0,2,0.000000,0.158508,1,1,13.030303,6,1.3030
683,5,4,0,14,10,4,1,14,4,0.500000,38,1.0,0,0.947368,0.360000,2,8,6.400000,1,0.8000
706,18,1,0,13,20,3,13,0,11,0.294118,860,1.0,0,0.000000,0.916667,0,0,46.388889,8,3.1111
619,8,3,0,10,33,8,0,10,8,0.428571,164,1.0,1,0.965517,0.300000,2,5,19.375000,16,2.7500


In [7]:
x_test

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
711,52,4,8,46,84,1030,28,22,45,0.851307,658,0.833333,4,0.616000,0.145105,2,9,11.423077,10,1.3846
499,3,1,0,3,11,0,2,1,3,0.500000,64,1.000000,0,0.000000,0.555556,0,0,19.666667,1,0.6667
44,11,2,4,12,20,0,6,7,11,0.200000,88,1.000000,1,0.750000,0.227273,1,7,6.909091,3,1.3636
238,7,1,0,29,17,0,26,3,7,0.000000,155,1.000000,0,0.000000,0.464286,0,0,21.000000,4,1.8571
237,2,1,0,2,3,0,2,0,2,0.000000,29,0.000000,0,0.000000,0.875000,0,0,12.000000,0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,3,3,0,2,11,3,2,0,3,2.000000,61,0.000000,0,0.950000,0.833333,1,2,19.333333,5,2.3333
176,6,4,0,36,48,15,10,30,5,2.000000,282,0.000000,0,0.938272,0.400000,2,12,46.000000,8,3.6667
527,14,4,0,9,40,0,1,8,13,0.076923,174,1.000000,1,0.941704,0.295918,2,3,11.357143,1,0.9286
19,3,5,0,3,4,3,1,2,3,2.000000,8,0.000000,0,0.990698,0.666667,1,2,1.666667,1,0.6667


In [8]:
y_train = train['bug']
y_test = test['bug']

In [9]:
y_train

245    0
344    0
473    0
282    0
51     0
      ..
531    0
683    0
706    0
619    0
224    0
Name: bug, Length: 578, dtype: int64

In [10]:
y_test

711    1
499    0
44     0
238    0
237    0
      ..
81     0
176    0
527    0
19     0
184    0
Name: bug, Length: 145, dtype: int64

In [11]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [12]:
x_train_np

array([[ 7.        ,  2.        ,  0.        , ..., 17.42857143,
         6.        ,  1.8571    ],
       [ 4.        ,  3.        ,  0.        , ...,  9.75      ,
         1.        ,  0.75      ],
       [ 2.        ,  2.        ,  0.        , ..., 18.5       ,
         0.        ,  0.        ],
       ...,
       [18.        ,  1.        ,  0.        , ..., 46.38888889,
         8.        ,  3.1111    ],
       [ 8.        ,  3.        ,  0.        , ..., 19.375     ,
        16.        ,  2.75      ],
       [10.        ,  3.        ,  0.        , ...,  3.8       ,
         1.        ,  0.9       ]])

In [13]:
x_test_np

array([[52.        ,  4.        ,  8.        , ..., 11.42307692,
        10.        ,  1.3846    ],
       [ 3.        ,  1.        ,  0.        , ..., 19.66666667,
         1.        ,  0.6667    ],
       [11.        ,  2.        ,  4.        , ...,  6.90909091,
         3.        ,  1.3636    ],
       ...,
       [14.        ,  4.        ,  0.        , ..., 11.35714286,
         1.        ,  0.9286    ],
       [ 3.        ,  5.        ,  0.        , ...,  1.66666667,
         1.        ,  0.6667    ],
       [42.        ,  1.        ,  0.        , ...,  9.23809524,
         6.        ,  1.1905    ]])

In [14]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [15]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [16]:
train_x

array([[[ 7.        ,  2.        ,  0.        , ..., 17.42857143,
          6.        ,  1.8571    ]],

       [[ 4.        ,  3.        ,  0.        , ...,  9.75      ,
          1.        ,  0.75      ]],

       [[ 2.        ,  2.        ,  0.        , ..., 18.5       ,
          0.        ,  0.        ]],

       ...,

       [[18.        ,  1.        ,  0.        , ..., 46.38888889,
          8.        ,  3.1111    ]],

       [[ 8.        ,  3.        ,  0.        , ..., 19.375     ,
         16.        ,  2.75      ]],

       [[10.        ,  3.        ,  0.        , ...,  3.8       ,
          1.        ,  0.9       ]]])

In [17]:
test_y

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 1,
       0, 4, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])

In [18]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [19]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
19/19 [==============================] - 4s 14ms/step - loss: 0.4502 - mse: 0.4502 - mae: 0.2321 - root_mean_squared_error: 0.6710 - mean_squared_logarithmic_error: 0.1242
Epoch 2/100
19/19 [==============================] - 0s 5ms/step - loss: 0.4431 - mse: 0.4431 - mae: 0.2418 - root_mean_squared_error: 0.6657 - mean_squared_logarithmic_error: 0.1202
Epoch 3/100
19/19 [==============================] - 0s 5ms/step - loss: 0.4358 - mse: 0.4358 - mae: 0.2542 - root_mean_squared_error: 0.6601 - mean_squared_logarithmic_error: 0.1165
Epoch 4/100
19/19 [==============================] - 0s 5ms/step - loss: 0.4259 - mse: 0.4259 - mae: 0.2666 - root_mean_squared_error: 0.6526 - mean_squared_logarithmic_error: 0.1117
Epoch 5/100
19/19 [==============================] - 0s 5ms/step - loss: 0.4153 - mse: 0.4153 - mae: 0.2859 - root_mean_squared_error: 0.6445 - mean_squared_logarithmic_error: 0.1079
Epoch 6/100
19/19 [==============================] - 0s 5ms/step - loss: 0.4038 - ms

19/19 [==============================] - 0s 5ms/step - loss: 0.1464 - mse: 0.1464 - mae: 0.1506 - root_mean_squared_error: 0.3826 - mean_squared_logarithmic_error: 0.0396
Epoch 89/100
19/19 [==============================] - 0s 5ms/step - loss: 0.1446 - mse: 0.1446 - mae: 0.1570 - root_mean_squared_error: 0.3803 - mean_squared_logarithmic_error: 0.0421
Epoch 90/100
19/19 [==============================] - 0s 5ms/step - loss: 0.1508 - mse: 0.1508 - mae: 0.1569 - root_mean_squared_error: 0.3884 - mean_squared_logarithmic_error: 0.0429
Epoch 91/100
19/19 [==============================] - 0s 5ms/step - loss: 0.1439 - mse: 0.1439 - mae: 0.1519 - root_mean_squared_error: 0.3793 - mean_squared_logarithmic_error: 0.0412
Epoch 92/100
19/19 [==============================] - 0s 5ms/step - loss: 0.1374 - mse: 0.1374 - mae: 0.1443 - root_mean_squared_error: 0.3707 - mean_squared_logarithmic_error: 0.0382
Epoch 93/100
19/19 [==============================] - 0s 5ms/step - loss: 0.1490 - mse: 0.149

In [20]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 3750), started 3:14:19 ago. (Use '!kill 3750' to kill it.)

In [21]:
score = model.evaluate(test_x, test_y)
score

5/5 [==============================] - 1s 2ms/step - loss: 0.2452 - mse: 0.2452 - mae: 0.2429 - root_mean_squared_error: 0.4952 - mean_squared_logarithmic_error: 0.0887


[0.24519161880016327,
 0.24519161880016327,
 0.24293923377990723,
 0.4951682686805725,
 0.08870325237512589]